In [7]:
import cv2, os, glob, shutil
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output, Video
from tqdm.auto import tqdm

# Move dir

In [4]:
folder_list = glob.glob('../data/crawling_img/*')
normal_dir = '../data/binary/normal'
flood_dir = '../data/binary/flood'
temp_normal = '../data/binary/temp_normal'
temp_flood = '../data/binary/temp_flood'
print(folder_list)

['../data/crawling_img/침수 cctv', '../data/crawling_img/車の洪水', '../data/crawling_img/침수', '../data/crawling_img/überflutetes Auto', '../data/crawling_img/flooding road', '../data/crawling_img/Überschwemmungen auf der Straße', '../data/crawling_img/flooded car', '../data/crawling_img/道路浸水', '../data/crawling_img/flooded', '../data/crawling_img/flooding', '../data/crawling_img/inundación de coches', '../data/crawling_img/被淹的车']


In [21]:
def move_dir(source_dir, normal_dir, flood_dir):
    # 크롤링으로 모은 이미지들을 이진분류에 쓰기위해 분류하는 작업
    source = source_dir.split('/')[-1]
    img_list = glob.glob(source_dir+'/*.jpg')
    n = len(img_list)
    for i,img_path in enumerate(img_list):
        print('{}/{}'.format(i,n))
        img_name = img_path.split('/')[-1]
        img = plt.imread(img_path)
        plt.imshow(img)
        plt.axis('off')
        plt.show()
        
        decision = input('정상 : 0, flood : 1, 삭제 : d, 사용안함 : n, quit: q \n')
        
        if decision=='0' :
            shutil.move(img_path, os.path.join(temp_normal, source+'_'+img_name))
        elif decision=='1' :
            shutil.move(img_path, os.path.join(temp_flood, source+'_'+img_name))
        elif decision=='d':
            os.remove(img_path)
        elif decision=='q' :
            break
        else:
            pass
        clear_output()

In [24]:
# 0, -3
move_dir(folder_list[0], normal_dir, flood_dir)

# CCTV 이미지를 프레임으로 분리

In [76]:
def get_frame(video_path, save_dir):
    video = cv2.VideoCapture(video_path)
    video_name = video_path.split('/')[-1]
    if not video.isOpened():
        print("Could not Open :", video_name)
        exit(0)
        
#     length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
#     width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
#     height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

#     print("length :", length)
#     print("width :", width)
#     print("height :", height)
    print("fps :", fps)
    
    count = 0
    while(video.isOpened()):
        ret, image = video.read()
#         if int(video.get(1)) % 20 == 0 :
        if(int(video.get(1)) % round(fps)*5 == 0): #앞서 불러온 fps 값을 사용하여 1초마다 추출
            try :
                print('Saved frame number :', str(int(video.get(1))))
                cv2.imwrite(save_dir + "/frame{}_{}.jpg".format(count, video_name), image)
                print('Saved frame%d.jpg' % count)
                count+=1 
            except Exception as e:
                print(e)
                print(image)
        if ret is False:
            break
            
    video.release()
    clear_output()

In [75]:
video_list = glob.glob('../data/video/*day_1024.mp4')
print(len(video_list))
#print(video_list)
# play mp4 on jupyter notebook
Video(video_list[1])

125


In [77]:
## Execute the Function
for video in video_list:
    get_frame(video, '../data/video_capture')

In [89]:
len(os.listdir('../data/video_capture/'))

2305

In [88]:
#del_list = glob.glob('../data/video_capture/*_18_day_1024*.jpg')
del_list = glob.glob('../data/video_capture/*.jpg')
for i, d in enumerate(del_list):
    if i % 9 == 0 :
        os.remove(d)

In [5]:
# for later use
def get_one_frame(video_path, save_dir):
    video = cv2.VideoCapture(video_path)
    
    if not video.isOpened():
        print("Could not Open :", filepath)
        exit(0)
        
    ret, image = video.read()
    try :
        file_name = video_path.split('/')[-1]
        cv2.imwrite(save_dir + "/{}.jpg".format(file_name), image)
    except Exception as e:
        video.release()
        print('Some thing wrong!')
        print(e)
        
    video.release()
#     plt.imshow(image)
#     plt.axis('off')
#     plt.show()

for video in video_list:
    get_one_frame(video, '../data/video_capture')

# 이진 분류를 위해 크롤링한 이미지 위치 이동

In [79]:
img_list = glob.glob('../data/binary/도로_cctv/*.jpg')
print(len(img_list))
print(flood_dir)
print(normal_dir)

123


In [81]:
flooded = ['177','191','250','340','400','406','422','636','638','661','730','72','762','785']
for img_path in img_list :
    num = img_path.split('/')[-1].split('.')[0]
    if num in flooded :
        shutil.move(img_path, os.path.join(flood_dir, num+'.jpg'))
    else:
        shutil.move(img_path, os.path.join(normal_dir, num+'.jpg'))
img_list = glob.glob('../data/binary/도로_cctv/*.jpg')
print(len(img_list))

0


In [13]:
## need to refactor##
# TODO
path = '../data/binary/'
src = '../data/binary/normal'
img_list = glob.glob(path+'*.jpg')
print(img_list)
print(len(img_list))
for img in img_list:
    img_name = img.split('/')[-1]
    new_path = os.path.join(src, '도로'+img_name)
    shutil.move(img, new_path)
    print(new_path)

['../data/binary/871.jpg', '../data/binary/828.jpg', '../data/binary/9689.jpg']
3
../data/binary/normal/도로871.jpg
../data/binary/normal/도로828.jpg
../data/binary/normal/도로9689.jpg


# 이진 분류를 위한 이미지 데이터 개수 확인

In [90]:
normal = glob.glob(normal_dir+'/*.jpg')
print('normal :', len(normal))
flood = glob.glob(flood_dir+'/*.jpg')
print('flood :', len(flood))

normal : 7723
flood : 6868
